# The University of Melbourne, School of Computing and Information Systems
# COMP90049 Introduction to Machine Learning, 2020 Semester 1
-----
## Project 1: Understanding Student Success with Naive Bayes
-----
###### Student Name(s): Xu Shi
###### Python version: 3.7.2
###### Submission deadline: 11am, Wed 22 Apr 2019

This iPython notebook is a template which you will use for your Project 1 submission. 

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [251]:
# This function should open a data file in csv, and transform it into a usable format 
def load_data():
    import pandas as pd
    data = pd.read_csv('student.csv', sep=',')
    return data

In [252]:
# This function should split a data set into a training set and hold-out test set
def split_data(data, test_size):
    """
    split the data into train set and test set
    :param data: Dtype from pd.read_csv
    :param test_size: float, define the position to split
    :return:
    """
    import numpy as np
    X = data[list(data.columns[:-1])].values # get the instances matrix
    y = data['Grade'] # get the class vector
    index = np.arange(data.shape[0]) # get the number of the dataset
    np.random.shuffle(index) # shuffle the order of the data
    X = X[index] # reorder the instances matrix
    y = y[index] # reorder the class vector

    split_point = int(X.shape[0] * test_size) # define the position to split the data into train and test
    X_train, X_test = X[:split_point], X[split_point:] 
    y_train, y_test = y[:split_point], y[split_point:]

    return X_train, X_test, y_train, y_test

In [253]:
# This function should build a supervised NB model
def train(X, y, alpha):
    """
    train or generate the probability matrix of Naive Bayes Classifier
    :param X: Dtype from pd.read_csv, train set
    :param y: Dtype from pd.read_csv, train class
    :param alpha: Laplace smooth index
    :return:
    """
    y_class_count = {}
    feature_dimension = len(X[1]) # number of feature

    # get the number of each labels
    for c in y:
        y_class_count[c] = y_class_count.get(c, 0) + 1 # generate the dict of class, e.g. {'A':'69',...}
    y_class_tuple = sorted(y_class_count.items(), reverse=False) # generate the tuple of class and sort it in terms of number, e.g. [('A','69'),...]
    K = len(y_class_tuple)         # the specific number of class grade
    N = len(y)                     # the number of instances

    # get the prior probability
    prior_prob = {}
    for key in range(len(y_class_tuple)):
        prior_prob[y_class_tuple[key][0]] = (y_class_tuple[key][1] + alpha) / (N + K * alpha)  # laplace smooth
    
    # get the value set of each feature
    feature_value = []  # feature with different value
    feature_value_number = []  # the number of unique values of each feature
    for feature in range(feature_dimension):
        unique_feature = list(set(X[:, feature])) # use `set` to get the unique value
        feature_value_number.append(len(unique_feature))
        feature_value.append(unique_feature)
    
    # calculate the conditional probability
    conditional_prob = []
    # calculate the count (x = a & y = c)
    for j in range(feature_dimension):
        count = [[0 for i in range(len(y_class_count))] for i in range(feature_value_number[j])] 
        # use list comprehension to generate zero matrix, (feature_value_number[j] rows x y_class_count cols) 
        for i in range(len(X[:, j])):
            for k in range(len(feature_value[j])):
                for t in range(len(y_class_count)):
                    if X[:, j][i] == feature_value[j][k] and list(y)[i] == y_class_tuple[t][0]:
                        # x = value and y = class, get the count 
                        count[k][t] += 1
        # calculate the conditional probability
        for m in range(len(y_class_tuple)):
            for r in range(len(count)):
                count[r][m] = (count[r][m] + alpha) / (y_class_tuple[m][1] + alpha * feature_value_number[j])  # laplace smoothing
        conditional_prob.append(count)
    return y_class_tuple, prior_prob, feature_value, feature_value_number, conditional_prob

In [254]:
def classify(y_class_tuple, prior_prob, feature_value, conditional_prob, feature_value_number, alpha, instance):
    """
    generate the answer of classification
    :param y_class_tuple: list, the tuple of class and sort it in terms of number
    :param prior_prob: float list, prior probability of class
    :param feature_value: list, feature value of all the attributes
    :param conditional_prob: float list, posterior probability
    :param feature_value_number: float list, number of different unique features
    :param alpha: float, Laplace smooth index default 1
    :param instance: list, one row of test set
    :return:
    """
    import math
    predict = {}
    for m in range(len(y_class_tuple)):
        # get the prior_probability of m-th label in y_class_tuple
        yhat = math.log(prior_prob[y_class_tuple[m][0]]) # use log-transformation to avoid float missing
        for n in range(len(instance)):
            if instance[n] in feature_value[n]:
                index = feature_value[n].index(instance[n]) # locate the feature in feature_value
                yhat = yhat + math.log(conditional_prob[n][index][m]) # accumulate the probability
            else:
                # if the value of feature is not in training set, return the laplace smoothing
                yhat = alpha / (feature_value_number[n] * alpha)
        predict[y_class_tuple[m][0]] = yhat
    return predict

In [255]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(y_class_tuple, prior_prob, feature_value, feature_value_number, conditional_prob, X, alpha, flag=0):
    """
    predict the class for an instance or a set of instances, based on a trained model 
    :param y_class_tuple: list, the tuple of class and sort it in terms of number
    :param prior_prob: float list, prior probability of class
    :param feature_value: list, feature value of all the attributes
    :param conditional_prob: float list, posterior probability
    :param feature_value_number: float list, number of different unique features
    :param alpha: float, Laplace smooth index default 1
    :param X: Dtype from pd.read_csv, test set
    :param flag: set 1 return probability or set 0 return prediction, default 0
    :return:
    """
    import operator as op
    test_num = len(X)
    prediction = [0 for i in range(test_num)]
    probability = [0 for i in range(test_num)]
    for i in range(test_num):
        result = classify(y_class_tuple, prior_prob, feature_value, conditional_prob, feature_value_number, 1, X[i, :])
        # result is the probability of each class
        result = sorted(result.items(), key=op.itemgetter(1), reverse=True) # the max probability is the predict class
        prediction[i] = result[0][0] # show the predict answer
        probability[i] = result[0][1] # show the predict probability

    if flag:
        return probability
    else:
        return prediction

In [256]:
# This function should evaluate a set of predictions in terms of accuracy
def evaluate(p, y_test):
    accuracy = sum(p == y_test)/len(y_test)
    return accuracy

## Main Process

In [257]:
data = load_data()
X_train, X_test, y_train, y_test = split_data(data, 0.7)
y_class_tuple, prior_prob, feature_value, feature_value_number, conditional_prob = train(X_train, y_train, 1)
p = predict(y_class_tuple, prior_prob, feature_value, feature_value_number, conditional_prob, X_test, 1)
evaluate(p, y_test)

0.4358974358974359

## Questions (you may respond in a cell or cells below):

You should respond to Question 1 and two additional questions of your choice. A response to a question should take about 100–250 words, and make reference to the data wherever possible.

### Question 1: Naive Bayes Concepts and Implementation

- a Explain the ‘naive’ assumption underlying Naive Bayes. (1) Why is it necessary? (2) Why can it be problematic? Link your discussion to the features of the students data set. [no programming required]
- b Implement the required functions to load the student dataset, and estimate a Naive Bayes model. Evaluate the resulting classifier using the hold-out strategy, and measure its performance using accuracy.
- c What accuracy does your classifier achieve? Manually inspect a few instances for which your classifier made correct predictions, and some for which it predicted incorrectly, and discuss any patterns you can find.

### Question 2: A Closer Look at Evaluation

- a You learnt in the lectures that precision, recall and f-1 measure can provide a more holistic and realistic picture of the classifier performance. (i) Explain the intuition behind accuracy, precision, recall, and F1-measure, (ii) contrast their utility, and (iii) discuss the difference between micro and macro averaging in the context of the data set. [no programming required]
- b Compute precision, recall and f-1 measure of your model’s predictions on the test data set (1) separately for each class, and (2) as a single number using macro-averaging. Compare the results against your accuracy scores from Question 1. In the context of the student dataset, and your response to question 2a analyze the additional knowledge you gained about your classifier performance.

### Question 3: Training Strategies 

There are other evaluation strategies, which tend to be preferred over the hold-out strategy you implemented in Question 1.
- a Select one such strategy, (i) describe how it works, and (ii) explain why it is preferable over hold-out evaluation. [no programming required]
- b Implement your chosen strategy from Question 3a, and report the accuracy score(s) of your classifier under this strategy. Compare your outcomes against your accuracy score in Question 1, and explain your observations in the context of your response to question 3a.

### Question 4: Model Comparison

In order to understand whether a machine learning model is performing satisfactorily we typically compare its performance against alternative models. 
- a Choose one (simple) comparison model, explain (i) the workings of your chosen model, and (ii) why you chose this particular model. 
- b Implement your model of choice. How does the performance of the Naive Bayes classifier compare against your additional model? Explain your observations.

### Question 5: Bias and Fairness in Student Success Prediction

As machine learning practitioners, we should be aware of possible ethical considerations around the
applications we develop. The classifier you developed in this assignment could for example be used
to classify college applicants into admitted vs not-admitted – depending on their predicted
grade.
- a Discuss ethical problems which might arise in this application and lead to unfair treatment of the applicants. Link your discussion to the set of features provided in the students data set. [no programming required]
- b Select ethically problematic features from the data set and remove them from the data set. Use your own judgment (there is no right or wrong), and document your decisions. Train your Naive Bayes classifier on the resulting data set containing only ‘unproblematic’ features. How does the performance change in comparison to the full classifier?
- c The approach to fairness we have adopted is called “fairness through unawareness” – we simply deleted any questionable features from our data. Removing all problematic features does not guarantee a fair classifier. Can you think of reasons why removing problematic features is not enough? [no programming required]


# Answer to Question 1:
- a
    - (1) Naive Bayes method is based on Bayes algorithm, which assumes that the attributes are independent of each other when the target value is given. That is to say, no attribute variable has a large proportion in the decision-making result, and no attribute variable has a small proportion in the decision-making result. Although this simplification method reduces the classification effect of Bayesian classification algorithm to a certain extent, it greatly simplifies the complexity of Bayesian method in practical application scenarios.
    - (2) Naive Bayesian model assumes that the attributes are independent from each other, which is often not true in practical application. When the number of attributes is relatively large or the correlation between attributes is large, the classification effect is not good. For example, the correlation between the attribute `Dalc` and attribute `Walc` is very large in the `student.csv`.
- b: See the code above.
- c
    - (1) The average accuracy between 0.35 and 0.4.
    - (2) For example, for those students who absent from school more than ten times can be easy to be predict to 'D' or 'F'.

# Answer to Question 2:
- a
    - (i) For tasks with very uneven distribution of positive and negative cases, the `accuracy` is not effective. From the perspective of prediction results, `precision` describes how many of the positive examples predicted by the two classifiers are real positive examples, that is, how many of the positive examples predicted by the two classifiers are accurate. `recall` describes how many of the real positive examples in the test set are selected by the two classifiers, that is, how many of the real positive examples are recalled by the two classifiers. When the two indicators conflict, we need `F1-measure`.
    - (ii) The advantage of accuracy rate is simple calculation, easy to understand / can be used for two classification or multi classification. The disadvantage is that when the data is unbalanced, it can not measure the quality of the classifier well. `Precision` refers to the probability of correct prediction of positive samples in the prediction results. `Recall` the recall rate, indicates the probability of being correctly predicted as a positive sample in the original sample. `F1-measure` represents the harmonic average evaluation index of precision and recall.
    - (iii) If the number of samples in each class is similar, the difference between macro average and micro average is not significant. If the difference between each class is large and you want to pay more attention to the class with large sample size: use macro average. If the micro average is much lower than the macro average, the class with large sample size should be checked.
- b
    - (1)(2)

In [335]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
class_names = ['A', 'A+', 'B', 'C', 'D', 'F']
print("acc:%f"%(accuracy_score(y_test, p)))
print(confusion_matrix(y_test, p))
print(classification_report(y_test, p, target_names=class_names))

acc:0.435897
[[ 5  0  3  5  0  0]
 [ 0  0  1  0  2  0]
 [ 6  0  9  9  8  3]
 [ 6  1  7 20  6  2]
 [ 0  1  1 15 32 20]
 [ 1  1  0  1 11 19]]
              precision    recall  f1-score   support

           A       0.28      0.38      0.32        13
          A+       0.00      0.00      0.00         3
           B       0.43      0.26      0.32        35
           C       0.40      0.48      0.43        42
           D       0.54      0.46      0.50        69
           F       0.43      0.58      0.49        33

   micro avg       0.44      0.44      0.44       195
   macro avg       0.35      0.36      0.35       195
weighted avg       0.45      0.44      0.43       195



Obviously, micro average is better than macro average in model classification. Because of the large number of classes C, D and F, their `precision` and `recall` are very large.

# Answer to Question 5:
- a The development of technology needs the limitation of ethics and rules. After all, it is these basic regulations that keep human society moving forward in constant friction, collision and conflict. In this case, classifier maybe arrange those students who parents education are low to D or F. This kind of discrimination is unreasonable and unacceptable. After all, parents' education level can't be linked with children's classes.
- b

In [341]:
data = load_data()
data = data[data.columns[15:]]
X_train, X_test, y_train, y_test = split_data(data, 0.7)
y_class_tuple, prior_prob, feature_value, feature_value_number, conditional_prob = train(X_train, y_train, 1)
p = predict(y_class_tuple, prior_prob, feature_value, feature_value_number, conditional_prob, X_test, 1)
evaluate(p, y_test)

0.3230769230769231

It may work better in the imagination, but it turns out that the results are not very different. It can only be said that ethical considerations cannot be calculated rationally on the level of pure mathematics.

- c: The core of this idea is to ignore the label of discrimination, but it will lead to a one size fits all problem, that is to say, it will lead all students to pull together to run.